In [1]:
# Required libraries 

import glob
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

#Employment cube raw data files 

empCube = 'C:/Users/604070/Desktop/Files/Performance/OPM_HR_Analytics/Model_Development/Data/EmpCube/FACTDATA_MAR201?.TXT'

#Reference tables source files  

refData = 'C:/Users/604070/Desktop/Files/Performance/OPM_HR_Analytics/Model_Development/Data/REF_Tables/*'

#accensions and seperations raw data files

accData = 'C:/Users/604070/Desktop/Files/Performance/OPM_HR_Analytics/Model_Development/Data/ACC/ACCDATA_FY2011-2017.TXT'
sepData = 'C:/Users/604070/Desktop/Files/Performance/OPM_HR_Analytics/Model_Development/Data/SEP/SEPDATA_FY2011-2017.TXT'


def load_emp_fact():

    '''Load OPM Employment Cube data
       Output: Return processed df of 2018 data'''

    df_total = pd.DataFrame()

    for name in glob.glob(empCube):


        df = pd.read_csv(name,dtype={ 'AGYSUB': str, 'LOC': str, 'AGELVL': str, 'EDLVL': str, 'GSEGRD': str, 'LOSLVL': str, 'OCC': str, 'PATCO': str,
       'PPGRD': str, 'SALLVL': str, 'STEMOCC': str, 'SUPERVIS': str, 'TOA': str, 'WORKSCH': str, 'WORKSTAT': str,
       'DATECODE': str, 'EMPLOYMENT': str, 'SALARY': str, 'LOS': float})    
        df_total = pd.concat([df_total, df])

    #clean salary col cast to integer
    df_total['SALARY'] = df_total['SALARY'].str.replace('$','')
    df_total['SALARY'] = df_total['SALARY'].str.replace(',','')

    df_total = df_total.dropna(axis = 0, how = 'any')

    df_total['SALARY'] = df_total['SALARY'].astype(int)
    
    df_total.drop(['EDLVL', 'EMPLOYMENT', 'STEMOCC', 'SUPERVIS', 'WORKSTAT'], axis = 1 ,inplace = True)
    
    #just 2018 data 
    df_2018 = df_total.loc[df_total['DATECODE'] == '201803']
    
    df_2018_date = df_2018.drop(['DATECODE'], axis =1)
    
    return df_2018_date


def load_acc_fact():

    '''Load accessions data
    
       Output: returns clean dataframe of all accensions data'''

    print('Loading data...')

    df = pd.read_csv(accData, low_memory=False)

    df = df.dropna(axis = 0, how = 'any')

    df['SALARY'] = df['SALARY'].astype(int)
    
    return df    

def load_sep_fact():

    '''Load separations data
    
       Output: returns clean dataframe of all seperations data'''

    df = pd.read_csv(sepData, low_memory=False)

    df = df.dropna(axis = 0, how = 'any')

    df['SALARY'] = df['SALARY'].astype(int)
    
    df = df[df['OCC'] != '****']

    df = df[df['TOA'] != '**']

    return df




def load_dim_table():

    '''load reference tables
    
    Output: returns a list containing dataframes of each reference table '''

    ref_list = []

    for name in glob.glob(refData):

        ref_list.append(pd.read_csv(name))

    return ref_list  



def join_func( df, df_other):
    
    '''Input: two inputs... df a given dataframe ... df_other a given dataframe to be joined to df
       
       Output: Returns a dataframe containing containing both input dataframes '''

    df = df.merge(df_other)

    return df


def joins(df, data):
    
    '''Inputs: df- dataframe to join reference tables; data - flag used to specify which refrence tables to join
    
       Output: Returns dataframe joined to reference tables'''
    
    #indexing based on number and order of reference tables found in the directory

    dim = load_dim_table()

    acc = dim[0]
    age = dim[1]
    agency = dim[2]
    education = dim[4]
    date = dim[5]
    grade = dim[6][1:]
    location = dim[7]
    los = dim[8]
    occ = dim[9]
    patco = dim[10]
    paygroup = dim[11]
    salary = dim[12]
    separation = dim[13]
    stem = dim[14]
    supervisor = dim[15]  
    toa = dim[16]
    workstat = dim[17]
    worksch = dim[18]

    # load fact table 

    df_join = df

    '''reference tables joined to acc/sep fact tables'''  

    if (data == 'accessions'):

        df_join = join_func(df_join, acc)
        
        df_join = join_func(df_join, date)
        
        df_join = join_func(df_join, patco)
    
    elif(data =='separations'):

        df_join = join_func(df_join, separation)
        
        df_join = join_func(df_join, date)
        
        df_join = join_func(df_join, patco)
    
    else:
        
        pass
    

    df_join = join_func(df_join, age)

    df_join = join_func(df_join, agency)    

    df_join = join_func(df_join, location)

    df_join = join_func(df_join, los)

    df_join = join_func(df_join, paygroup)

    df_join = join_func(df_join, salary)

    df_join = join_func(df_join, toa)

    df_join = join_func(df_join, worksch)

    df_join = join_func(df_join, occ)

    return df_join



def join_dim_tables():   

    '''Reduce accension and separation data in star schema to dataframe
    
       Output: returns the accessions and separations data joined to all relevant reference tables'''


    df_joins_acc = joins(load_acc_fact(), 'accessions')

    df_joins_sep = joins(load_sep_fact(), 'separations')


    return df_joins_acc, df_joins_sep  


def raw_data():
    
    '''Load complete accension and separations data '''
    
    acc, sep = join_dim_tables()

    rawData = pd.concat([acc, sep], axis = 0, sort = True)
    
    rawData['EFDATE'] = rawData['EFDATE'].astype(str)

    rawDatalabel = rawData.loc[:,('ACCT','SEPT')]

    rawDatalabel['ACCT'].update(rawDatalabel.pop('SEPT'))
    
    rawDatalabel['labels'] = rawDatalabel['ACCT'].apply(lambda x: 1 if x == 'Quit' else 0)

    data = pd.concat([rawData, rawDatalabel['labels']], axis = 1 )

    data.drop(['ACC','LOSLVL','TOATYP','AGELVL','OCCTYP','PPTYP', 'PPTYPT', 'SEPT', 'AGY', 'AGYSUB', 'AGYTYP', 'EFDATE', 'FY', 'LOC', 'LOCTYP', 'OCC', 'OCCFAM','PAYPLAN','PPGROUP','WORKSCH', 'WSTYP','SALLVL','SEP', 'TOA', 'QTR', 'PATCO'  ], axis =1, inplace=True)

    return data, rawDatalabel.ACCT
    

def pre_process():
    
    data = raw_data()
    
    data = data.iloc[:,1:]
    data['Year'] = data['EFDATET'].apply(lambda x: x[4:])
    data['Month'] = data['EFDATET'].apply(lambda x: x[:2])
    data['QTRT'] = data['QTRT'].apply(lambda x: x[:7])
    
    data_dummy = pd.get_dummies(data[['AGELVLT', 'AGYSUBT', 'AGYT', 'AGYTYPT', 'COUNT', 'FYT',
       'GSEGRD', 'LOCT', 'LOCTYPT', 'LOS', 'LOSLVLT', 'OCCFAMT', 'OCCT',
       'OCCTYPT', 'PATCOT', 'PAYPLANT', 'PPGRD', 'PPGROUPT', 'QTRT', 'SALARY',
       'SALLVLT', 'TOAT', 'TOATYPT', 'WORKSCHT', 'WSTYPT', 'ACCT',
       'Year', 'Month']])
    
    whole = pd.concat([data_dummy, data['EFDATET']], axis = 1)
    # 2017 data removed for testing data 
    test_data = whole[whole['Year_2017'] == 1]
    
    train_data = whole[whole['Year_2017'] != 1]
    
    test_data = test_data.drop(['Year_2017'], axis = 1)
    
    train_data = train_data.drop(['Year_2017'], axis = 1)
    
    return train_data, test_data


def normalize(df):
    
    df = df.groupby('EFDATET').sum()
    
    for col in list(df.columns):
       
        mean, std = df[col].mean(), df[col].std()

        df.loc[:, col] = (df[col] -mean) /(std + 1) 
        
    
        
    y = df['ACCT'].reset_index().iloc[:,1]
    
    y = y.values
    
    X = df.drop(['ACCT'], axis = 1).reset_index().iloc[:,1:]
    
    X = X.values
        
    return X, y, df 
    


In [2]:
data, verbal_labels = raw_data()

Loading data...


In [3]:
acc, sep = join_dim_tables()
print(acc.shape)
print(sep.shape)

Loading data...
(1034751, 51)
(1038378, 51)


In [4]:
data.head()

,ACCT,AGELVLT,AGYSUBT,AGYT,AGYTYPT,COUNT,EFDATET,FYT,GSEGRD,LOCT,...,PPGRD,PPGROUPT,QTRT,SALARY,SALLVLT,TOAT,TOATYPT,WORKSCHT,WSTYPT,labels
0,New Hire - Excepted Service Appointment,25-29,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...,AA-ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,Small Independent Agencies (less than 100 empl...,1,APR 2016,FY 2016,12,11-DISTRICT OF COLUMBIA,...,GS-12,Standard GSEG Pay Plans,APR-JUN 2016,77490,"$70,000 - $79,999",30-Excepted Service - Schedule A,Permanent,F - Full-time Nonseasonal,Full-time,0
1,New Hire - Excepted Service Appointment,25-29,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...,AA-ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,Small Independent Agencies (less than 100 empl...,1,JUN 2016,FY 2016,12,11-DISTRICT OF COLUMBIA,...,GS-12,Standard GSEG Pay Plans,APR-JUN 2016,77490,"$70,000 - $79,999",30-Excepted Service - Schedule A,Permanent,F - Full-time Nonseasonal,Full-time,0
2,New Hire - Excepted Service Appointment,25-29,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...,AA-ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,Small Independent Agencies (less than 100 empl...,1,APR 2012,FY 2012,12,11-DISTRICT OF COLUMBIA,...,GS-12,Standard GSEG Pay Plans,APR-JUN 2012,79864,"$70,000 - $79,999",30-Excepted Service - Schedule A,Permanent,F - Full-time Nonseasonal,Full-time,0
3,New Hire - Excepted Service Appointment,30-34,AA00-ADMINISTRATIVE CONFERENCE OF THE UNITED S...,AA-ADMINISTRATIVE CONFERENCE OF THE UNITED STATES,Small Independent Agencies (less than 100 empl...,1,MAR 2017,FY 2017,12,11-DISTRICT OF COLUMBIA,...,GS-12,Standard GSEG Pay Plans,JAN-MAR 2017,79720,"$70,000 - $79,999",30-Excepted Service - Schedule A,Permanent,F - Full-time Nonseasonal,Full-time,0
4,New Hire - Excepted Service Appointment,25-29,BG00-PENSION BENEFIT GUARANTY CORPORATION,BG-PENSION BENEFIT GUARANTY CORPORATION,Medium Independent Agencies (100-999 employees),1,OCT 2010,FY 2011,12,11-DISTRICT OF COLUMBIA,...,GS-12,Standard GSEG Pay Plans,OCT-DEC 2010,74872,"$70,000 - $79,999",30-Excepted Service - Schedule A,Permanent,F - Full-time Nonseasonal,Full-time,0


In [5]:
verbal_labels.value_counts()

New Hire - Competitive Service Appointment    524340
New Hire - Excepted Service Appointment       423002
Quit                                          348819
Retirement - Voluntary                        265756
Termination (Expired Appt/Other)              237338
Transfer Out - Individual Transfer            129573
Transfer In - Individual Transfer              78592
Retirement - Disability                        14720
Death                                          14481
Retirement - Early Out                         11194
Transfer In - Mass Transfer                     8817
Transfer Out - Mass Transfer                    8806
Retirement - Other                              5002
Reduction In Force (RIF)                        2510
Other Separation                                 179
Name: ACCT, dtype: int64

In [6]:
data.labels.value_counts()

0    1724310
1     348819
Name: labels, dtype: int64

In [9]:
test = data[data.labels == 1] 

In [11]:
test.shape

(348819, 28)

In [13]:
data.shape

(2073129, 28)